# VacationPy

---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data


In [59]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [60]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wacheng,33.7833,114.5167,272.59,40,0,1.92,CN,2024-12-26
1,1,weno,7.4515,151.8468,301.66,80,81,9.41,FM,2024-12-26
2,2,igarka,67.4667,86.5833,253.55,98,100,5.59,RU,2024-12-26
3,3,adamstown,-25.0660,-130.1015,297.82,74,11,4.34,PN,2024-12-26
4,4,waitangi,-43.9535,-176.5597,289.72,99,100,2.24,NZ,2024-12-26


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.


In [61]:
# %%capture --no-display
# Configure the map plo
city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Max Temp",
    size = "Humidity",
    colorbar=True,
    title="City Latitude vs. Longitude",
    xlabel="Longitude",
    ylabel="Latitude",
    grid=True,
    height=500,
    width=700,
    cmap="coolwarm",
    tiles= True,
    geo = True
)
# Display the map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Max Temp,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition


In [62]:
# Narrow down cities that fit criteria and drop any results with null values
desired_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 273+27)
    & (city_data_df["Max Temp"] > 273+21)
    & (city_data_df["Max Temp"] > 70)
    & (city_data_df["Wind Speed"] < 10)
    & (city_data_df["Cloudiness"] <20)
]

# Drop any rows with null values
city_data_df = city_data_df.dropna()
# Display sample data
desired_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,adamstown,-25.0660,-130.1015,297.82,74,11,4.34,PN,2024-12-26
14,14,nova sintra,14.8667,-24.7167,297.13,73,4,4.97,CV,2024-12-26
17,17,callao,-12.0667,-77.1500,299.10,60,0,5.66,PE,2024-12-26
20,20,bamboo flat,11.7000,92.7167,298.65,77,16,4.66,IN,2024-12-26
45,45,cabo san lucas,22.8909,-109.9124,298.32,53,0,3.09,MX,2024-12-26


### Step 3: Create a new DataFrame called `hotel_df`.


In [63]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = desired_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,adamstown,PN,-25.0660,-130.1015,74,
14,nova sintra,CV,14.8667,-24.7167,73,
17,callao,PE,-12.0667,-77.1500,60,
20,bamboo flat,IN,11.7000,92.7167,77,
45,cabo san lucas,MX,22.8909,-109.9124,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.


In [64]:
# Set parameters to search for a hotel


radius = 10000
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    query_url = base_url+f"?categories=accommodation&bias={params['bias']}&filter={params['filter']}&apiKey={geoapify_key}"
    name_address = requests.get(query_url)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
nova sintra - nearest hotel: Kaza Di Zaza
callao - nearest hotel: Casa Píos
bamboo flat - nearest hotel: Hotel Megapode Nest
cabo san lucas - nearest hotel: Comfort Rooms
carnarvon - nearest hotel: Carnarvon Motel
asaita - nearest hotel: ናርዶስ ሆቴል
pisco - nearest hotel: No hotel found
ixtapa - nearest hotel: Casa Menta
anloga - nearest hotel: Lumana House
takoradi - nearest hotel: Hillcrest
port elizabeth - nearest hotel: Waterford Hotel
tecuala - nearest hotel: No hotel found
kerikeri - nearest hotel: Kerikeri Homestead Motel
harper - nearest hotel: Cape Palmas Investment
inhambane - nearest hotel: Hotel de Inhambane
dassa-zoume - nearest hotel: Auberge Doguessi
cidreira - nearest hotel: Hotel Castelo
tabou - nearest hotel: hôtel le rochet
juan griego - nearest hotel: Hotel Descanso Tropical
robore - nearest hotel: Lajas
vryburg - nearest hotel: International Hotel
purranque - nearest hotel: Pensión y Restaurant Estrella
u

,City,Country,Lat,Lng,Humidity,Hotel Name
3,adamstown,PN,-25.0660,-130.1015,74,No hotel found
14,nova sintra,CV,14.8667,-24.7167,73,Kaza Di Zaza
17,callao,PE,-12.0667,-77.1500,60,Casa Píos
20,bamboo flat,IN,11.7000,92.7167,77,Hotel Megapode Nest
45,cabo san lucas,MX,22.8909,-109.9124,53,Comfort Rooms
64,carnarvon,AU,-24.8667,113.6333,32,Carnarvon Motel
68,asaita,ET,11.5636,41.4394,69,ናርዶስ ሆቴል
69,pisco,PE,-13.7000,-76.2167,73,No hotel found
94,ixtapa,MX,20.7000,-105.2000,69,Casa Menta
123,anloga,GH,5.7947,0.8973,83,Lumana House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.


In [65]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    colorbar=True,
    title="Hotel Locations",
    hover_cols=["Hotel Name", "Country"],
    xlabel="Longitude",
    ylabel="Latitude",
    grid=True,
    height=500,
    width=700,
    cmap="coolwarm",
    tiles=True,
    geo=True
)

# Display the map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)